In [10]:
from transformers import BertForSequenceClassification, BertTokenizer, AdamW, BertConfig
import torch
import pandas as pd
from tqdm import tqdm
from peft import PeftModel

In [11]:
# 读取测试数据
test_df = pd.read_csv("./data/test/NSL-KDD-10000.csv")
# test_df = pd.read_csv("./data/test/KDD99-10000.csv")
# test_df = pd.read_csv("./data/test/UNSW_NB15-10000.csv")
# test_df = pd.read_csv("./data/test/X-IIoTID-10000.csv")
# test_df = pd.read_csv("./data/test/NSL-KDD-Gamma.csv")
# test_df = pd.read_csv("./data/test/NSL-KDD-Gaussian.csv")
# test_df = pd.read_csv("./data/test/NSL-KDD-Laplace.csv")
# test_df = pd.read_csv("./data/test/NSL-KDD-Poisson.csv")
# test_df = pd.read_csv("./data/test/NSL-KDD-Uniform.csv")
# test_df = pd.read_csv("./data/test/NSL-KDD-Cauchy.csv")
texts = test_df["flow"].tolist()

In [12]:
config = BertConfig.from_pretrained("./test_trainer-sft-lora/checkpoint-12500/config.json")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", config=config)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
# # 加载基础模型和LoRA适配器
# base_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
# model = PeftModel.from_pretrained(base_model, "./test_trainer-sft-lora/checkpoint-12500")
# # model = PeftModel.from_pretrained(base_model, "./test_trainer-sft-diy-lora-mix/checkpoint-95000")
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
f:\Python3.10\lib\site-packages\peft\config.py:162: UserWarning: Unexpected keyword arguments ['_name_or_path', 'architectures', 'attention_probs_dropout_prob', 'classifier_dropout', 'gradient_checkpointing', 'hidden_act', 'hidden_dropout_prob', 'hidden_size', 'initializer_range', 'intermediate_size', 'layer_norm_eps', 'max_position_embeddings', 'model_type', 'num_attention_heads', 'num_hidden_layers', 'pad_token_id', 'position_embedding_type', 'problem_type', 'torch_dtype', 'transformers_version', 'type_vocab_size', 'use_cache', 'vocab_size'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library an

In [13]:
# 遍历数据，找到最长文本的长度
text_lengths = [len(tokenizer(text)["input_ids"]) for text in texts]
max_length = max(text_lengths)
max_length = min(max(text_lengths), 500)
max_length

123

In [14]:
def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")

In [15]:
# 推理配置
model.eval()
torch.set_grad_enabled(False)
device = torch.device("cuda")
model.to(device)
# 批量预测（带进度条）
batch_size = 16
predictions = []
for i in tqdm(range(0, len(texts), batch_size), 
              desc="预测进度", 
              unit="batch",
              bar_format="{l_bar}{bar:20}{r_bar}"):
    batch_texts = texts[i:i+batch_size]
    inputs = tokenize_function(batch_texts)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    outputs = model(**inputs)
    batch_preds = torch.argmax(outputs.logits, dim=-1).cpu().tolist()
    predictions.extend(batch_preds)


预测进度: 100%|████████████████████| 625/625 [01:15<00:00,  8.30batch/s]


In [16]:
# 计算指标
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
if "class" in test_df.columns:
    true_labels = test_df["class"].tolist()
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='binary')  # 二分类使用binary
    recall = recall_score(true_labels, predictions, average='binary')
    f1 = f1_score(true_labels, predictions, average='binary')
    print(f"\n模型在测试集上的指标:")
    print(f"准确率 (Accuracy): {accuracy:.4f}")
    print(f"精确率 (Precision): {precision:.4f}")
    print(f"召回率 (Recall): {recall:.4f}")
    print(f"F1 分数 (F1-score): {f1:.4f}")


模型在测试集上的指标:
准确率 (Accuracy): 0.5440
精确率 (Precision): 0.5502
召回率 (Recall): 0.8730
F1 分数 (F1-score): 0.6750
